In [1]:
from json import loads
from pyspark.sql import SparkSession
import warnings
import pandas as pd
warnings.filterwarnings("ignore")
from pyspark.sql.functions import col,from_json,udf,split,explode,lit,array
from pyspark.ml.feature import NGram
from pyspark.sql.types import StructType,StructField, StringType, IntegerType,MapType,FloatType,ArrayType
import numpy as np
from sklearn.metrics import classification_report,accuracy_score,precision_score,recall_score,f1_score

In [2]:
spark = SparkSession.\
        builder.\
        appName("ml").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "1024m").\
        getOrCreate()

23/01/16 16:57:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
data = spark.read.parquet('hdfs://namenode:9000/TikiCleaned/Comment')
data.createOrReplaceTempView('data')
df = spark.sql("""
    select distinct clean_content,rating,sentiment,
    case
        when rating >= 4 then 2
        when rating = 3 then 1
        else 0
    end as label
    from data
    where clean_content <> '' and clean_content is not null and clean_content <> ' '
""")

In [4]:
data.count()

3769857

In [5]:
df.count()

1087002

In [6]:
df.groupby('label').count().show()

+-----+------+
|label| count|
+-----+------+
|    1| 82636|
|    2|861369|
|    0|142997|
+-----+------+



In [9]:
df_test = spark.read.parquet('hdfs://namenode:9000/ml/test_data')
df_train = spark.read.parquet('hdfs://namenode:9000/ml/train_data')

In [10]:
df_test.count()

4977

In [11]:
df_train.count()

318847

In [13]:
df_test.groupby('true_label').count().show()

+----------+-----+
|true_label|count|
+----------+-----+
|         0|  805|
|         1|  538|
|         2| 3634|
+----------+-----+



In [14]:
df_train.groupby('label').count().show()

+-----+------+
|label| count|
+-----+------+
|    1| 17382|
|    2|262509|
|    0| 38956|
+-----+------+

